# Real-time face detection with Edge TPU

Let's build a real-time face detection system with Edge TPU!

In [1]:
import numpy as np
import cv2
from time import time
from edgetpu.detection.engine import DetectionEngine
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont

Let's load our model and setup the capture device (usb camera):

In [2]:
engine = DetectionEngine('data/mobilenet_ssd_v2_face_quant_postprocess_edgetpu.tflite')
font = ImageFont.truetype("/usr/share/fonts/truetype/freefont/FreeMonoBold.ttf", 20, encoding="unic")
cap = cv2.VideoCapture(0)

Let's perform face detection! For each captured frame: a) we convert it to the appropriate format (BGR to RGB), b) perform the face detection as before, c) annotate the frame and d) show the frame back!

In [3]:
 while (True):

        # Run inference.
        start_time = time()

        # Get the frame
        _, frame = cap.read()

        # Convert to PIL image
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_im = Image.fromarray(rgb_frame)
        draw = ImageDraw.Draw(pil_im)

        # Run the detection
        start_time_inference = time()
        ans = engine.DetectWithImage(pil_im, threshold=0.1, keep_aspect_ratio=True, relative_coord=False)
        
        end_time_inference = time()
        # Get the results
        if ans:
            for obj in ans:
                box = obj.bounding_box.flatten().tolist()
                draw.rectangle(box, outline='red', width=4)
        end_time = time()
        
        # Calculate some statistics
        fps = 1. / (end_time - start_time)
        fps_inf = 1. / (end_time_inference - start_time_inference)
        draw.text((0, 0), 'FPS:  %3.2f , Inference FPS:  %3.2f' % (fps, fps_inf), font=font)

        bgr_frame = cv2.cvtColor(np.array(pil_im), cv2.COLOR_RGB2BGR)

        # Display the frame
        cv2.imshow('frame', bgr_frame)
        
        # Quit on q
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

In [4]:
cap.release()
cv2.destroyAllWindows()